In [23]:
from data import Dataset
from training import train_pythae_model

%load_ext autoreload
%autoreload 2

import torch

from metrics import NDCG_binary_at_k_batch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
class dotdict(dict):
        """dot.notation access to dictionary attributes"""
        __getattr__ = dict.get
        __setattr__ = dict.__setitem__
        __delattr__ = dict.__delitem__

In [25]:
args = {}

args = dotdict(args)
args['data'] = 'gowalla' #'foursquare'

args['model'] = 'MultiCIWAE' #SimCLR_reco_model, MultiVAE, BYOL_reco_model

args['n_epoches'] = 100

args['n_epoches_dec'] = 50

args['lrdec'] = 1e-3

args['lrenc'] = 1e-3

args['train_batch_size'] = 1024

args['val_batch_size'] = 200

args['n_val_samples'] = 1

args['annealing'] = True

args.total_anneal_steps = 50

args.anneal_cap = 1.

args.print_info_ = 5

args.metric = NDCG_binary_at_k_batch

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

args.n_views = 2

args.temperature = .07

args.dropout_rate = 0.83

args.criterion_dec = torch.nn.CrossEntropyLoss()

args.m = 0.995

In [26]:
dataset = Dataset(args)
# pdb.set_trace()
layers = [100, 600, dataset.n_items]
args.z_dim = layers[0]
args.l2_coeff = 0.

In [27]:
next(dataset.next_train_batch()).shape

torch.Size([1024, 21208])

In [28]:
from other_models import *
from pythae.models import VAMPConfig, VAEConfig, VAE_LinNF_Config, VAE_IAF_Config
from pythae.models.nn import BaseEncoder, BaseDecoder
from pythae.models.base.base_utils import ModelOutput
from models import make_linear_network


class Encoder(BaseEncoder):
    def __init__(self, dims):
        BaseDecoder.__init__(self)
        self.layers = make_linear_network(
            dims,
            encoder=True
        )
        self.dims = dims

    def forward(self, x):
        out = self.layers(x)
        return ModelOutput(
            embedding=out[:, :self.dims[-1]],
            log_covariance=out[:, self.dims[-1]:]
            )
    
class Decoder(BaseDecoder):
    def __init__(self, dims):
        BaseDecoder.__init__(self)
        self.layers = make_linear_network(
            dims,
            encoder=False
        )
        self.dims = dims
        
    def forward(self, x):
        out = self.layers(x)
        return ModelOutput(
            reconstruction=out[:self.dims[-1]]
            )

In [29]:
if args.model == "MultiVAE":
    from other_models import MultiVAEPythae
    from pythae.models import VAEConfig

    config = VAEConfig(
        latent_dim=args.z_dim,
        input_dim=(26150,)
    )

    model = MultiVAEPythae(
        model_config=config,
        encoder=Encoder(dims=layers[::-1]),
        decoder=Decoder(dims=layers)
    ).to(args.device)

elif args.model == "MultiVAMP":
    from other_models import MultiVAMP
    from pythae.models import VAMPConfig

    config = VAMPConfig(
        latent_dim=args.z_dim,
        input_dim=(26150,),
        n_components=500
    )

    model = MultiVAMP(model_config=config,
        encoder=Encoder(dims=layers[::-1]),
        decoder=Decoder(dims=layers)
    ).to(args.device)

elif args.model == "MultiVAELinNF":
    from other_models import MultiVAELinNF
    from pythae.models import VAE_LinNF_Config

    config = VAE_LinNF_Config(
        latent_dim=args.z_dim,
        input_dim=(26150,),
        flows=['Planar', 'Radial', 'Planar', 'Radial', 'Planar']
    )

    model = MultiVAELinNF(model_config=config,
        encoder=Encoder(dims=layers[::-1]),
        decoder=Decoder(dims=layers)
    ).to(args.device)

elif args.model == "MultiVAEIAF":
    from other_models import MultiVAEIAF
    from pythae.models import VAE_IAF_Config

    config = VAE_IAF_Config(
        latent_dim=args.z_dim,
        input_dim=(26150,),
        n_made_blocks=3
    )

    model = MultiVAEIAF(model_config=config,
        encoder=Encoder(dims=layers[::-1]),
        decoder=Decoder(dims=layers)
    ).to(args.device)

elif args.model == "MultiCIWAE":
    from other_models import MultiCIWAE
    from pythae.models import CIWAEConfig

    config = CIWAEConfig(
        latent_dim=args.z_dim,
        input_dim=(21208,),
        number_samples=5,
        beta=0.5
    )

    model = MultiCIWAE(model_config=config,
        encoder=Encoder(dims=layers[::-1]),
        decoder=Decoder(dims=layers)
    ).to(args.device)

metric_values = train_pythae_model(model, dataset, args)

  0%|          | 0/100 [00:00<?, ?it/s]

1109.2017


/home/clement/Documents/recommender_systems_experiments/src/metrics.py:32: RuntimeWarning: invalid value encountered in divide
  a = DCG / IDCG
  0%|          | 0/100 [00:11<?, ?it/s]

Best NDCG: 0.08965297658064632
Current NDCG: 0.08965297658064632


Error: You must call wandb.init() before wandb.log()

In [71]:
from pythae.models import AutoModel

model = AutoModel.load_from_folder("/home/clement/Documents/recommender_systems_experiments/models/best_model_MultiVAE_data_gowalla_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None_learntransitions_False_initstepsize_0.005")

In [21]:
config.tes = 1.
config

CIWAEConfig(name='CIWAEConfig', input_dim=(26150,), latent_dim=100, uses_default_encoder=False, uses_default_decoder=False, reconstruction_loss='mse', number_samples=5, beta=0.5)

In [72]:
model

VAE(
  (decoder): Decoder(
    (layers): Sequential(
      (0): Linear(in_features=200, out_features=600, bias=True)
      (1): Tanh()
      (2): Linear(in_features=600, out_features=21208, bias=True)
    )
  )
  (encoder): Encoder(
    (layers): Sequential(
      (0): Linear(in_features=21208, out_features=600, bias=True)
      (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Tanh()
      (3): Linear(in_features=600, out_features=400, bias=True)
    )
  )
)

In [81]:
torch.load('../models/best_model_MultiVAELinNF_data_gowalla_K_None_N_None_learnreverse_False_anneal_False_lrdec_0.001_lrenc_None_learntransitions_False_initstepsize_0.005.pt')

RuntimeError: PytorchStreamReader failed locating file data.pkl: file not found